In [5]:
import pandas as pd
import numpy as np

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [7]:
df = pd.read_csv("/content/tested.csv")

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Steps we plan to follow:


*   impute missing values in age and embarked.
*   ohe - sex and embarked
*   scaling
*   feature selection
*   train model using decision tree



In [9]:
df.drop(columns =['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                    df['Survived'],
                                                    test_size = 0.2,
                                                    random_state = 42)

In [11]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


In [12]:
y_train.sample(5)

317    0
64     0
263    1
334    0
96     1
Name: Survived, dtype: int64

In [13]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy ='most_frequent'),[6])
], remainder ='passthrough')

# here 2/6 represents index value of the column

In [14]:
# one hot encoder
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse = False, handle_unknown='ignore'), [1,6])

], remainder = 'passthrough')

In [15]:
# Scaling

trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

In [16]:
# feature selection
trf4 = SelectKBest(score_func = chi2, k=5)

In [17]:
#train the model
trf5 = DecisionTreeClassifier()

# **Create Pipeline**

In [18]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

# **Pipeline VS make_pipeline**
pipeline requires naming of steps, make_pipelines does not.

(same applies to ColumnTransformer vs make_column_transformer)

In [19]:
# Alternate syntax
pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)

In [20]:
# train
pipe.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7f1b9a058a60>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# **Explore the Pipeline**

In [21]:
# pipeline ki functionalities ki help se alg alg transformers ki info nikali jaa skti hai

# pipe.named_steps
# pipe.named_step['trf1]
# pipe.named_step['trf1].transformers
# pipe.named_step['trf1].transformers_[0]

In [22]:
# to display the pipeline

from sklearn import set_config
set_config(display='diagram')

In [23]:
# predict

y_pred = pipe.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5952380952380952

# **Cross validation using pipeline**

In [25]:
# cross validation using cross_val_score using pipeline

from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring ='accuracy').mean()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

0.622840343735866

# **Grid Search using pipeline**

In [26]:
# gridsearchcv - for hyper parameter tuning

params = {
    'trf5__max_depth' : [1,2,3,4,5,None]
}



In [27]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring = 'accuracy')
grid.fit(X_train, y_train)

ValueError: ignored

In [28]:
# grid.best_score_

In [29]:
# grid.best_params_

# **Exporting the Pipeline**

In [30]:
# export

import pickle
pickle.dump(pipe, open('pipe.pkl','wb'))